<a href="https://colab.research.google.com/github/Fimbrez/PersonalProjects/blob/main/WineRatings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd


In [ ]:
from pyspark.sql import SparkSession

# read csv file into pyspark dataframe
infile = 'winemag-data-130k-v2.csv'

spark = SparkSession \
    .builder \
    .appName("Winemag Wine") \
    .getOrCreate()


In [ ]:
df = spark.read.csv(infile, inferSchema=True, header = True)

In [ ]:
type(df)

In [ ]:
#showing df schema; see all the variables we will be using
df.printSchema()

In [ ]:
#Look at data
df.show(3)

# Linear Regression


### Price as a predictor


In [ ]:
#subsetting the data to prepare it for linear regression
vars_to_keep = ["points","price"]

# subset the dataframe on these predictors
df2 = df.select(vars_to_keep)
df2.show(3)

In [ ]:
#importing libararies we will be using
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler 
from pyspark.mllib.linalg import Vectors

In [ ]:
#Renaming the response variables
df2 = df2.withColumnRenamed("points","Ratings")

In [ ]:
df2.show(3)

In [ ]:
#Removing null values
df2 = df2.filter(df2.price.isNotNull())
df2 = df2.filter(df2.Ratings.isNotNull())

In [ ]:
#Looking at df2 now
df2.show(3)

In [ ]:
#Typecasting the variables to integers
from pyspark.sql.types import IntegerType
df2 = df2.withColumn("Ratings", df2["Ratings"].cast("float"))
df2 = df2.withColumn("price", df2["price"].cast("float")) #needs to be of type array

In [ ]:
assembler = VectorAssembler(inputCols=["price"], outputCol="features") 
df2 = assembler.transform(df2.na.drop())

In [ ]:
#dropping the price column
df2 = df2.drop("price")

In [ ]:
df2.show(20)

In [ ]:
from pyspark.sql.functions import col

In [ ]:
# Split data approximately into training (60%) and test (40%)
seed = 314
train_test = [0.6, 0.4]
train_data, test_data = df2.randomSplit(train_test, seed)

In [ ]:
train_data.show(3)

In [ ]:
#checking count of training and test to match up with total
(train_data.count(), test_data.count(), df2.count())

In [ ]:
from pyspark.ml.regression import LinearRegression # note this is from the ML package

maxIter=10
regParam=0.3
elasticNetParam=0.8

lr = LinearRegression(featuresCol = 'features', labelCol = 'Ratings', maxIter = maxIter, 
                      regParam = regParam, elasticNetParam = elasticNetParam)

In [ ]:
model1 = lr.fit(train_data)

In [ ]:
lrPred = model1.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol = "Ratings", predictionCol = "prediction",
                          metricName = "rmse")
mse = eval.evaluate(lrPred, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

### Variety, Winery, and Country as predictors

In [ ]:
#subsetting the data to prepare it for linear regression
vars_to_keep = ["variety","winery","country","points"]

# subset the dataframe on these predictors
df3 = df.select(vars_to_keep)
df3.show(3)

#### Encoding the string-valued data

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").setHandleInvalid("skip").fit(df3) for column in list(set(df3.columns)-set(['points'])) ]


pipeline = Pipeline(stages=indexers)
df3 = pipeline.fit(df3).transform(df3)

df3.show(3)

In [ ]:
#Renaming the response variables
df3 = df3.withColumnRenamed("points","Ratings")
df3 = df3.withColumn("Ratings", df3["Ratings"].cast("float"))
df3 = df3.withColumn("variety_index", df3["variety_index"].cast("float"))
df3 = df3.withColumn("winery_index", df3["winery_index"].cast("float"))
df3 = df3.withColumn("country_index", df3["country_index"].cast("float"))

In [ ]:
# dropping the string-valued columns
df3 = df3.select(["Ratings","variety_index","winery_index","country_index"])
df3 = df3.filter(df3.variety_index.isNotNull())
df3 = df3.filter(df3.Ratings.isNotNull())
df3 = df3.filter(df3.country_index.isNotNull())
df3 = df3.filter(df3.winery_index.isNotNull())


In [ ]:
assembler = VectorAssembler(inputCols=["variety_index","winery_index","country_index"], outputCol="features") 
df3 = assembler.transform(df3.na.drop())
df3.show(3)

In [ ]:
df3 = df3.select("Ratings","features")
df3.show(3)

In [ ]:
# Feature scaling

# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled", 
                                withStd=True, withMean=False)

# Fit the DataFrame to the scaler; this computes the mean, standard deviation of each feature
scaler = standardScaler.fit(df3)

# Transform the data in `df2` with the scaler
scaled_df = scaler.transform(df3)
scaled_df.show(3)

In [ ]:
# Split data approximately into training (60%) and test (40%)
seed = 314
train_test = [0.6, 0.4]
train_data, test_data = df3.randomSplit(train_test, seed)

In [ ]:
model2 = lr.fit(train_data)

In [ ]:
lrPred2 = model2.transform(test_data)

In [ ]:
mse = eval.evaluate(lrPred2, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

### Price, Variety, Winery, and Country as predictors

In [ ]:
vars_to_keep = ["variety","winery","country","points","price"]

# subset the dataframe on these predictors
df4 = df.select(vars_to_keep)
df4.show(3)

In [ ]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").setHandleInvalid("skip").fit(df4) for column in list(set(df4.columns)-set(['points','price'])) ]


pipeline = Pipeline(stages=indexers)
df4 = pipeline.fit(df4).transform(df4)

df4.show(3)

In [ ]:
df4 = df4.withColumnRenamed("points","Ratings")
df4 = df4.withColumn("Ratings", df4["Ratings"].cast("float"))
df4 = df4.withColumn("variety_index", df4["variety_index"].cast("float"))
df4 = df4.withColumn("winery_index", df4["winery_index"].cast("float"))
df4 = df4.withColumn("country_index", df4["country_index"].cast("float"))

In [ ]:
df4 = df4.select(["Ratings","variety_index","winery_index","country_index","price"])
df4 = df4.filter(df4.variety_index.isNotNull())
df4 = df4.filter(df4.Ratings.isNotNull())
df4 = df4.filter(df4.country_index.isNotNull())
df4 = df4.filter(df4.winery_index.isNotNull())
df4 = df4.filter(df4.price.isNotNull())

In [ ]:
df4 = df4.withColumn("price", df4["price"].cast("float"))

In [ ]:
assembler = VectorAssembler(inputCols=["variety_index","winery_index","country_index","price"], outputCol="features") 
df4 = assembler.transform(df4.na.drop())
df4.show(3)

In [ ]:
df4 = df4.select("Ratings","features")
df4.show(3)

In [ ]:
# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled", 
                                withStd=True, withMean=False)

# Fit the DataFrame to the scaler; this computes the mean, standard deviation of each feature
scaler = standardScaler.fit(df4)

# Transform the data in `df2` with the scaler
scaled_df = scaler.transform(df4)
scaled_df.show(3)

In [ ]:
train_data, test_data = df4.randomSplit(train_test, seed)

In [ ]:
model3 = lr.fit(train_data)

In [ ]:
lrPred3 = model3.transform(test_data)

In [ ]:
mse = eval.evaluate(lrPred3, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)